# etl.py test notebook
The goal of this notebook is to test the functions from `etl.py` independently

In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from etl import process_song_file,process_data

## 1. Testing `process_song_file`

In [2]:
conn = psycopg2.connect(
        "host=127.0.0.1 dbname=sparkifydb user=student password=student"
    )
cur = conn.cursor()

In [3]:
filepath='data/song_data'

In [7]:
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root, '*.json'))
    for f in files:
        all_files.append(os.path.abspath(f))

In [8]:
all_files[:3]

['/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAAW128F429D538.json']

In [ ]:
df = pd.read_json(all_files[0], lines=True)

# insert song record
song_data = (
    df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']]
    .astype(str)
    .tolist()
)
cur.execute(song_table_insert, song_data)

# insert artist record
artist_data = (
    df.loc[
        0,
        [
            'artist_id',
            'artist_name',
            'artist_location',
            'artist_latitude',
            'artist_longitude',
        ],
    ].values.tolist()
)

In [ ]:
process_song_file(cur, all_files[0])

In [ ]:
process_data(cur, conn, filepath='data/song_data', func=process_song_file)

## 2. Testing `create_database`

In [3]:
# connect to default database    
conn = psycopg2.connect(
    "host=127.0.0.1 dbname=studentdb user=student password=student"
)
conn.set_session(autocommit=True)
cur = conn.cursor()

In [ ]:
# create sparkify database with UTF8 encoding
cur.execute("DROP DATABASE IF EXISTS sparkifydb")

## 3. Testing `process_log_file`

In [5]:
filepath = 'data/log_data'

In [6]:
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root, '*.json'))
    for f in files:
        all_files.append(os.path.abspath(f))

In [7]:
all_files[:2]

['/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-25-events.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/log_data/2018/11/2018-11-27-events.json']

In [8]:
df = pd.read_json(all_files[2], lines=True, convert_dates=['ts'])
df = df[df['page'] == 'NextSong']

In [9]:
df.dtypes

artist                   object
auth                     object
firstName                object
gender                   object
itemInSession             int64
lastName                 object
length                  float64
level                    object
location                 object
method                   object
page                     object
registration            float64
sessionId                 int64
song                     object
status                    int64
ts               datetime64[ns]
userAgent                object
userId                   object
dtype: object

In [13]:
song_select_test = '''
SELECT s.song_id,
       s.artist_id
  FROM songs s
  JOIN artists a ON s.artist_id = a.artist_id
 WHERE 1 = 1
   AND s.title = %s
   AND a.name = %s
   AND s.duration = %s
'''

In [21]:
print(f'''
SELECT s.song_id,
       s.artist_id
  FROM songs s
  JOIN artists a ON s.artist_id = a.artist_id
 WHERE 1 = 1
   AND s.title = {row.song}
   AND a.name = {row.artist}
   AND s.duration = {row.length}
''')


SELECT s.song_id,
       s.artist_id
  FROM songs s
  JOIN artists a ON s.artist_id = a.artist_id
 WHERE 1 = 1
   AND s.title = Sehr kosmisch
   AND a.name = Harmonia
   AND s.duration = 655.77751



In [12]:
song_select

'\nSELECT\n    s.song_id, s.artist_id\nFROM\n    songs s\n        JOIN artists a ON s.artist_id = a.artist_id\nWHERE\n    s.title = %s\n    AND a.name = %s\n    AND s.duration = %s\n'

In [22]:
row = df.iloc[2]

In [24]:
row.song

'Marry Me'

In [25]:
row.artist

'Train'

In [26]:
row.length

205.45261

In [18]:
cur.execute(song_select_test, (row.song, row.artist, row.length))
results = cur.fetchone()

In [19]:
results

## Validation: Artist in logs and artist in Song file

### Artists in song_data

In [1]:
filepath='data/song_data'

In [4]:
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root, '*.json'))
    for f in files:
        all_files.append(os.path.abspath(f))

In [10]:
df_song_data = pd.DataFrame()
for file in all_files:
    df_song_data = pd.concat([df_song_data, pd.read_json(file, lines=True)])

In [13]:
df_song_data.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
0,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
0,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007


In [34]:
df_song_data[df_song_data.artist_name == 'Trafik']

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARKULSX1187FB45F84,39.49974,Utah,-111.54732,Trafik,424.12363,1,SOQVMXR12A81C21483,Salt In NYC,0


### Artists in log_data

In [19]:
all_files_log = []
for root, dirs, files in os.walk('data/log_data'):
    files = glob.glob(os.path.join(root, '*.json'))
    for f in files:
        all_files_log.append(os.path.abspath(f))

In [21]:
df_log_data = pd.DataFrame()
for file in all_files_log:
    df_log_data = pd.concat([df_log_data, pd.read_json(file, lines=True)])

In [23]:
df_log_data.shape

(8056, 18)

In [22]:
df_log_data.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,matchbox twenty,Logged In,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,Argue (LP Version),200,1543109954796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
1,The Lonely Island / T-Pain,Logged In,Jayden,F,1,Duffy,156.23791,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,I'm On A Boat,200,1543110131796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
2,None,Logged In,Jayden,F,2,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",GET,Home,1.540146e+12,846,None,200,1543110132796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
3,None,Logged In,Jayden,F,3,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",GET,Settings,1.540146e+12,846,None,200,1543110168796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76
4,None,Logged In,Jayden,F,4,Duffy,NaN,free,"Seattle-Tacoma-Bellevue, WA",PUT,Save Settings,1.540146e+12,846,None,307,1543110169796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76


In [39]:
row = df_log_data[df_log_data.artist == 'Trafik'].loc[240]
row

artist                                                      Trafik
auth                                                     Logged In
firstName                                               Jacqueline
gender                                                           F
itemInSession                                                    6
lastName                                                     Lynch
length                                                     380.212
level                                                         paid
location                         Atlanta-Sandy Springs-Roswell, GA
method                                                         PUT
page                                                      NextSong
registration                                           1.54022e+12
sessionId                                                      924
song                                                    Dirty Word
status                                                        

### Checking for unique Artists in each dataframe

In [32]:
set(df_song_data.artist_name.unique()).intersection(set(df_log_data.artist.unique()))

{'Blue Rodeo',
 'Elena',
 'Gob',
 'Gwen Stefani',
 'Jimmy Wakely',
 'Line Renaud',
 'Lionel Richie',
 'Lupe Fiasco',
 'Sophie B. Hawkins',
 'Tom Petty',
 'Trafik'}

In [41]:
len(df_log_data.artist.unique())

3149

In [42]:
len(df_song_data.artist_name.unique())

69

In [40]:
print(f'''
SELECT s.song_id,
       s.artist_id
  FROM songs s
  JOIN artists a ON s.artist_id = a.artist_id
 WHERE 1 = 1
   AND s.title = {row.song}
   AND a.name = {row.artist}
   AND s.duration = {row.length}
''')


SELECT s.song_id,
       s.artist_id
  FROM songs s
  JOIN artists a ON s.artist_id = a.artist_id
 WHERE 1 = 1
   AND s.title = Dirty Word
   AND a.name = Trafik
   AND s.duration = 380.21179

